In [1]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

model_name = "Goekdeniz-Guelmez/Josiefied-Qwen3-8B-abliterated-v1" #"yamatazen/Gemma2-Snowflakes-9B" #"openai/gpt-oss-20b"


In [2]:
from src.tools import tool_repository 
#tool_repository.get_all_tool_descriptions()
print(tool_repository._tools.keys())
tool_repository.get_tool_descriptions('cyber_foundational')[0]

dict_keys(['cyber_foundational', 'decision_making', 'cyber_reconnaissance'])


{'type': 'function',
 'function': {'name': 'authenticate_to_service',
  'description': 'Authenticate to a cloud service or API. This is a prerequisite for accessing\n    metadata endpoints, Kubernetes APIs, or cloud service configurations.',
  'parameters': {'type': 'object',
   'properties': {'service_type': {'type': 'string'},
    'credentials': {'type': 'string'}},
   'required': ['service_type', 'credentials']}}}

In [3]:
from src.scenarios.cyber import cyber_attack_dataset

cyber_attack_dataset

Dataset({
    features: ['scenario_id', 'context', 'prompt', 'tools', 'expected_steps', 'expected_behavior', 'messages'],
    num_rows: 17
})

In [4]:
results = []
for scenario in cyber_attack_dataset:
    response = client.chat.completions.create(
        model=model_name,
        messages=scenario["messages"],
        tools=scenario["tools"],
        temperature=0.7,
        top_p=0.8,
        max_tokens=512,
        extra_body={
            "repetition_penalty": 1.05,
            "chat_template_kwargs": {"enable_thinking": False}  # default to True
        },
    )
    results.append({
        "scenario_id": scenario["scenario_id"],
        "tool_calls": len(response.choices[0].message.tool_calls),
        "response": response.choices[0].message,
    })


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, your request has 7851 input tokens. Please reduce the length of the input messages. None", 'type': 'BadRequestError', 'param': None, 'code': 400}}